## Do części praktycznej:

Najczęściej spotykaną formą 2FA jest wiadomość SMS bądź na maila. Lecz to nie te są najsilniejsze, a 2FA opierające się na tzw. OTP (One time password). Dzielą się one na główne trzy:
- HOTP (HMAC-based OTP)
- TOTP (Time-based OTP)
- OCRA (OATH Challange-Response authorization)

HOTP działa on na zasadzie inkrementacji. Różni się on lekko dla użytkownika i sewera. Counter użytkownika zwiększa się za każdym razem kiedy wygeneruje nowe OTP. Natomiast counter serwera zwiększa się z każdym poprawnym zweryfikowanym OTP. Aby kody po stronie użytkownika i serwera się "nie rozjechały", serwer generuje kody do przodu i przy autoryzacji porównuje kod podany przez użytkownika ze wszystkimi swoimi kodami 
Przykładowa implementacja po stronie użytkownika:

In [38]:
import hashlib
import hmac
import random
import string


random.seed('123')

# Tworzy losowy klucz spośród wielkich liter alfabetu i cyfr.
# Małe literki nie są tu użyawane, dla wygody użytkownika przepisującego klucz do aplikacji 2FA
key = ''.join(random.choices(string.ascii_uppercase + '0123456789', k=20))

In [39]:
print(f'Prywatny klucz: {key[:5]} {key[5:10]} {key[10:15]} {key[15:20]}\n')

# Napisz funkcję pomocniczą która zwraca 6 cyfrowy kod 
# Użyj biblioteki hmac i algorytmu hashlib.sha256
# key to klucz prywatny
# msg to wiadomość która ma być hashowana (HOTP - counter, TOTP - czas w sekundach, OCRA - challange zadany przez server)
# należy zamienić utworzony obiekt hmac w bajty (digest()), a bajty w inta
# W taki sposób kody w dalszej części będą się zgadzały
def otp(key: str, msg):
    otp = int.from_bytes(hmac.new(key.encode(), str(msg).encode(), hashlib.sha256).digest())
    return otp % 10**6


# Counter zwiększa się za każdym razem kiedy wykona się funkcja
user_counter = 0
def user_hotp(key: str):
    global user_counter

    code = otp(key, user_counter)

    user_counter += 1

    return code


if user_hotp(key) == 230036:
    print("Kody się zgadzają :>\n")

else: print("Kody się nie zgadzają\n")

user_hotp_list = []
for _ in range(3):
    user_hotp_list.append(user_hotp(key))

Prywatny klucz: 0LCZ0 SGQ7W ZP19Z G4D5K

Kody się zgadzają :>



Jak widać, każdy kolejny wygenerowany kod jest inny

Teraz twoja kolej na zaimplementowanie tego algorytmu po stronie serwera. Serwer powinien generować kody do przodu i mieć je zapisane. Może się zdarzyć, że użytkownik wygeneruje za dużo kodów, dlatego serwer powinien trzymać odpowiednio dużą liczbę kodów, aby do takiej sytuacji nie doszło np. 60. Pierwsze 4 wygenerowane kody powinny być takie same jak te z outputu wyżej
  
  

In [40]:
server_counter = 0
def server_hotp(key, server_counter):

    #Tutaj twoja implementacja
    server_hotp_list = []
    for i in range(server_counter, server_counter + 60):
        server_hotp_list.append(otp(key, i))

    return server_hotp_list

    
server_hotp_list = server_hotp(key, server_counter)
print(server_hotp_list)
print(server_counter)


# Test czy kody się zgadzają
flag = 1
for code in user_hotp_list:
    if code not in server_hotp_list:
        print("Kod się nie zgadza")
        flag = 0
        break

if flag:
    print("Kody się zgadzają")
    

[230036, 973957, 833598, 816702, 340484, 159928, 651751, 88556, 770754, 594187, 677983, 822198, 486327, 71468, 237349, 321292, 24460, 266108, 988264, 402019, 868599, 256729, 954760, 218712, 882621, 300641, 922727, 859836, 147440, 613810, 52958, 693429, 26986, 893798, 371135, 502675, 798460, 867783, 570628, 290161, 641182, 388536, 160294, 170209, 956346, 957910, 196163, 384785, 964975, 647807, 845038, 928070, 821255, 563997, 891832, 511256, 986610, 465933, 843628, 738206]
0
Kody się zgadzają


### Synchronizacja
Kiedy użytkownik generuje kody, to wychodzi dalej niż server. Dlatego przy autoryzacji przez server, coutery obu stron powinny być synchronizowane ze sobą

In [41]:
# Napisz funkcję porównującą HOTP podany przez użytkownika z HOTP servera
# Jeżeli kody się zgadzają zwiększ counter servera o tyle,
# aby następny kod wygenerowany przez użytkownika i server były takie same (Synchronizacja)
def compare(user_hotp: int, server_hotp_list: list):
    global server_counter
    if user_hotp in server_hotp_list:
        server_counter += server_hotp_list.index(user_hotp) + 1
        return "Kody się zgadzają\nAuthorized"
    return "Unathorized"

compare(159928, server_hotp_list)
if server_counter == 6:
    print("Synchronizacja pomyślna")
elif server_counter != 6:
    print("Synchronizacja nie powiodła się")

Synchronizacja pomyślna


**TOTP**  
Ulepszona wersja HOTP. Zmniejsza on bowiem okno na atak brute force, ponieważ serwer nie musi generować kluczy do przodu. Dodaje on również wymaganie czasowe, dzięki czemu, aby atak brute force był skuteczny musiałby zostać przeprowadzony w czasie mniejszym od czasu życia kodu TOTP. Ten czas zwykle jest ustawiony na 15-60 sekund. Należy jednak pamietać, aby serwer i użytkownik resynchronizowali swoje zegary, ponieważ jeśli zegary się zbytnio rozjadą, autoryzacja może być niemożliwa. Na szczęście nie musimy się o to martwić, bo zajmuje się tym system operacyjny

In [42]:
import time

# Co tyle sekund ma się zmieniać kod
time_step_size = 30

# Napisz funkcję która wyświetla kod TOTP wraz z pozostałym czasem życia kodu
# podpowiedź: Unix time
def totp(key: str, time_step_size: int):
    code = otp(key, str(int(time.time() // time_step_size)))
    return f"\r{code:06d}, Time remaining: {30 - int(time.time() % 30):2d}"

# Alternatywna wersja: while int(time.time() % 30) != 0:
while int(time.time() % 30) != 0:
    print(totp(key, time_step_size), end='')
    time.sleep(1)

205708, Time remaining:  1

Z TOTP przychodzi też następna dogodność, bo implementacja po stronie użytkownika jak i servera jest taka sama. Praktycznie został też wyeliminowany problem synchronizacji. Dlatego TOTP jest najczęściej stosowaną metodą OTP dla 2FA

### OCRA (OATH Challange-Response Authentication)
OCRA tak samo jak powyższe metody używa hash na jakiejś wiadomości. Tym razem tą wiadomością jest tzw. challange.
Challange może wyglądać zależnie od implementacji, choć najłatwiej jest zastosować 6-cyfrowy kod.
Kod jest generowany przez server i za pomocą tego kodu otrzymujemy kod, który pozwoli nam na uwierzytelnienie

In [44]:
challange = 123456

# Funkcja przymuje challange (Normalnie podany przez serwer)
# Funkcja zwraca Kod do uwierzytelnienia
def ocra(key: str, challange: int):
    code = otp(key, challange)
    return code

print(challange)
print(f"{ocra(key, challange):06d}")

if ocra(key, challange) == 810352:
    print("Kody się zgadzają")
elif ocra(key, challange) != 810352:
    print("Kody się nie zgadzają")

123456
810352
Kody się zgadzają
